## Data_News_category Hackathon
### Classify data into 4 different categories

In [39]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\00001502\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\00001502\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\00001502\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\00001502\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
np.random.seed(216)

In [7]:
Corpus = pd.read_excel(r"Data_Train.xlsx",encoding='latin-1')

In [8]:
Corpus.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [9]:
# Step - a : Remove blank rows if any.
Corpus['STORY'].dropna(inplace=True)

In [10]:
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['STORY'] = [entry.lower() for entry in Corpus['STORY']]

In [11]:
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['STORY']= [word_tokenize(entry) for entry in Corpus['STORY']]

In [12]:
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['STORY']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [13]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['SECTION'],test_size=0.3)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [14]:
Train_Tfidf = Tfidf_vect.transform(Corpus['text_final'])
Target=Corpus['SECTION']

In [15]:
# print(Tfidf_vect.vocabulary_)

In [16]:
# print(Train_X_Tfidf)

# Modeling

# Naive Bayes

In [17]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  95.93709043250328


### ALL Models

In [18]:
%%time
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
# models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
# models.append(('RNDF', RandomForestClassifier()))
models.append(('NB', naive_bayes.MultinomialNB()))
models.append(('SVM', SVC(C=1.0, kernel='linear', degree=3,gamma='auto')))

# evaluate each model in turn
results = []
names = []
for name, model in models:
# 	kfold = model_selection.KFold(n_splits=10, random_state=seed)
# 	cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
# 	results.append(cv_results)
# 	names.append(name)
# 	msg = "%s: %f (%f)" % (name, round(cv_results.mean(),1), cv_results.std())
# 	print(msg)
    model.fit(Train_X_Tfidf,Train_Y)
    predictions_NB = model.predict(Test_X_Tfidf)
    # Use accuracy_score function to get the accuracy
    print("{} model Accuracy Score -> {} " .format(name,accuracy_score(predictions_NB, Test_Y)*100))

LR model Accuracy Score -> 96.54871122761031 
KNN model Accuracy Score -> 94.01485364788117 
CART model Accuracy Score -> 86.45696810834426 
NB model Accuracy Score -> 95.93709043250328 
SVM model Accuracy Score -> 96.94189602446484 
Wall time: 7.31 s


#### ADA boost on whole data

In [19]:
%%time
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, Train_Tfidf,Target, cv=5)
print(scores.mean())                           
# 0.9...
# 0.84399

0.8632723193104699
Wall time: 12.7 s


In [40]:
%%time
clf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(clf, Train_Tfidf,Target, cv=5)
print(scores.mean())  

0.9432368819778434
Wall time: 14.2 s


In [41]:
%%time
clf = GradientBoostingClassifier(n_estimators=100)
scores = cross_val_score(clf, Train_Tfidf,Target, cv=5)
print(scores.mean())  

0.9326172736658972
Wall time: 1min 46s


# Testing

### Manipulating the testing data

In [ ]:
Corpus_test = pd.read_excel(r"Data_Test.xlsx",encoding='latin-1')

Corpus_test.head()

In [ ]:
# Step - a : Remove blank rows if any.
Corpus_test['STORY'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus_test['STORY'] = [entry.lower() for entry in Corpus_test['STORY']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus_test['STORY']= [word_tokenize(entry) for entry in Corpus_test['STORY']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus_test['STORY']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus_test.loc[index,'text_final'] = str(Final_words)

Corpus_test.shape

In [ ]:
Train_X=Corpus['text_final']
Train_Y=Corpus['SECTION']
Test_X=Corpus_test['text_final']

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfi = Tfidf_vect.transform(Train_X)
Test_X_Tfi = Tfidf_vect.transform(Test_X)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['SECTION'],test_size=0.3)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
Train_X_Tfidf.shape,Train_Y.shape

# Have tried different Vectorizers to check if they improved the acuracy

# N-gram Vectorizer

In [23]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['SECTION'],test_size=0.3,random_state=216)

# Tfidf_vect = TfidfVectorizer(max_features=5000)
# Tfidf_vect.fit(Corpus['text_final'])
# Train_X_Tfidf = Tfidf_vect.transform(Train_X)
# Test_X_Tfidf = Tfidf_vect.transform(Test_X)

tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(Corpus['text_final'])
Train_X_Tfidf_ngram =  tfidf_vect_ngram.transform(Train_X)
Test_X_Tfidf_ngram =  tfidf_vect_ngram.transform(Test_X)


# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf_ngram,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf_ngram)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
# models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
# models.append(('RNDF', RandomForestClassifier()))
models.append(('NB', naive_bayes.MultinomialNB()))
models.append(('SVM', SVC(C=1.0, kernel='linear', degree=3,gamma='auto')))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    model.fit(Train_X_Tfidf_ngram,Train_Y)
    predictions_NB = model.predict(Test_X_Tfidf_ngram)
    # Use accuracy_score function to get the accuracy
    print("{} model Accuracy Score -> {} " .format(name,accuracy_score(predictions_NB, Test_Y)*100))

Naive Bayes Accuracy Score ->  85.75797291393621
LR model Accuracy Score -> 88.72870249017038 
KNN model Accuracy Score -> 37.96417649628659 
CART model Accuracy Score -> 78.02533857579729 
NB model Accuracy Score -> 85.75797291393621 
SVM model Accuracy Score -> 88.24814329401485 


# Character Level Vectorizer

In [24]:
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(Corpus['text_final'])
Train_X_Tfidf_chars =  tfidf_vect_ngram_chars.transform(Train_X) 
Test_X_Tfidf_chars =  tfidf_vect_ngram_chars.transform(Test_X) 

In [25]:
%%time
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
# models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
# models.append(('RNDF', RandomForestClassifier()))
models.append(('NB', naive_bayes.MultinomialNB()))
models.append(('SVM', SVC(C=1.0, kernel='linear', degree=3,gamma='auto')))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    model.fit(Train_X_Tfidf_chars,Train_Y)
    predictions_NB = model.predict(Test_X_Tfidf_chars)
    # Use accuracy_score function to get the accuracy
    print("{} model Accuracy Score -> {} " .format(name,accuracy_score(predictions_NB, Test_Y)*100))

LR model Accuracy Score -> 94.75753604193972 
KNN model Accuracy Score -> 69.59370904325033 
CART model Accuracy Score -> 76.84578418523373 
NB model Accuracy Score -> 85.4958497160332 
SVM model Accuracy Score -> 95.76234163390127 
Wall time: 1min 7s


# Different Vectorizers did not improve the accuracy of the model . I am sticking to my TFID vectorizer method and SVM model which has high accuracy.

# SVM

In [38]:
SVM = svm.SVC(C=1.1, kernel='linear', degree=4,gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

predictions_NB = Naive.predict(Test_X_Tfi)
print(predictions_NB)

submission = pd.DataFrame({
        "SECTION": predictions_NB
    })

submission.to_excel('submission_svm.xlsx', index=False)
submission = pd.read_excel('submission_svm.xlsx')
submission.head()

SVM Accuracy Score ->  96.59239842726082
[1 0 1 ... 2 2 2]


,SECTION
0,1
1,0
2,1
3,2
4,1


In [42]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['SECTION'],test_size=0.3,random_state=216)

SVM = svm.SVC(C=1.1, kernel='linear', degree=4,gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  30.53735255570118


In [43]:
%%time
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['SECTION'],test_size=0.3,random_state=10)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)


# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.1, kernel='linear', degree=4,gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  97.07295762341633
Wall time: 6.02 s


In [ ]:
216 96.41
162 96.63
1234 96.54
1234 92.2
7 96.76
0 96.02
10 97.02
# C
1.1 96.81  

10 1.1 97.02
# degree
2 97.02
3 97.07
4 97.07

## Parameter Tuning on SVM with grid search

## Tried different models with different paramenters and submitted different files for for verification. 

In [44]:
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [45]:
%%time
svc_param_selection(Train_X_Tfidf,Train_Y,5)

Wall time: 26min 44s


{'C': 0.001, 'gamma': 0.001}

In [46]:
SVM = svm.SVC(C=1, kernel='linear', degree=4,gamma=0.001)
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  31.367409349060726


In [47]:
# from sklearn.model_selection import svm, GridSearchCV
from sklearn.model_selection import GridSearchCV
def svc_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    degrees=[1,2,3,4]
    kernals=['rbf','linear']
    param_grid = {'C': Cs, 'gamma' : gammas,'degree':degrees}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
%%time
svc_selection(Train_Tfidf,Target,5)

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
model=GradientBoostingClassifier(random_state=216)
model.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = model.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("GradientBoostingClassifier Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

# Naive Bayes

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfi)
print(predictions_NB)

submission = pd.DataFrame({
        "SECTION": predictions_NB
    })

submission.to_excel('submission_nb.xlsx', index=False)
submission = pd.read_excel('submission_nb.xlsx')
submission.head(),submission.shape

In [85]:
predictions_NB = Naive.predict(Test_X_Tfi)
print(predictions_NB)

submission = pd.DataFrame({
        "SECTION": predictions_NB
    })

submission.to_excel('submission_nb.xlsx', index=False)
submission = pd.read_excel('submission_nb.xlsx')
submission.head()

submission = pd.DataFrame({
        "SECTION": predictions_NB
    })

submission.to_excel('submission_nb.xlsx', index=False)

submission = pd.read_excel('submission_nb.xlsx')
submission.head(),submission.shape

[1 2 1 ... 1 0 1]


(   SECTION
 0        1
 1        2
 2        1
 3        0
 4        1, (2748, 1))

# SVM

In [88]:

SVM = svm.SVC(C=1, kernel='linear', degree=4,gamma=0.001)
SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfi)

print((predictions_SVM))

print("SVM Accuracy Score -> ",accuracy_score(predictions_NB, predictions_SVM)*100)

submission = pd.DataFrame({
        "SECTION": predictions_SVM
    })

submission.to_excel('submission2.xlsx', index=False)

submission = pd.read_excel('submission2.xlsx')
submission.head(),submission.shape

[1 2 1 ... 1 0 1]
SVM Accuracy Score ->  97.0524017467249


(   SECTION
 0        1
 1        2
 2        1
 3        1
 4        1, (2748, 1))

In [89]:
predictions_NB.shape,predictions_SVM.shape,Corpus_test.shape

((2748,), (2748,), (2748, 2))

In [90]:
submission = pd.DataFrame({
    "Story":Corpus_test['STORY'],"SECTION_SVM": predictions_SVM,"SECTION_nb": predictions_NB
    })

submission.to_excel('checking.xlsx', index=False)